# Import Required Libraries and Functions
This section imports all necessary libraries and functions from the codebase for testing.

In [4]:
from sage.all import QQ, PolynomialRing, factorial
from basis import build_poly_from_exp, exp_from_monomial, compute_basis_exps
from reduction import sub_x_for_u, sub_y_for_v, reduce_poly
from differential import diff_from_exp_to_exp, diff_function
from matrix_and_cohom import matrix_of, torsion_exponents_from_elem_divs
from Frobenius import Frob_i, cap_deg, faster_Frob

base_ring = PolynomialRing(QQ, 'x,y,u,v,dx,dy')

ModuleNotFoundError: No module named 'utils'

# Test `build_poly_from_exp` and `exp_from_monomial`
Test the round-trip and correctness of polynomial construction and exponent extraction.

In [ ]:
# Test: build_poly_from_exp and exp_from_monomial
exp = [2, 1, 0, 2, 1, 0, 0]  # coef=2, x^1, y^0, u^2, v^1, dx^0, dy^0
poly = build_poly_from_exp(exp, R=base_ring)

# Check polynomial structure
x, y, u, v, dx, dy = base_ring.gens()
expected_poly = 2 * x**1 * v**1 / factorial(1) * u**2 / factorial(2)
assert poly == expected_poly, f"build_poly_from_exp failed: {poly} != {expected_poly}"

# Test round-trip: exp_from_monomial(build_poly_from_exp(...)) == exp
recovered_exp = exp_from_monomial(poly)
assert tuple(recovered_exp) == exp, f"exp_from_monomial round-trip failed: {recovered_exp} != {exp}"

# Test error on non-monomial
try:
    exp_from_monomial(base_ring(x + y))
    assert False, "exp_from_monomial should fail on non-monomial input"
except ValueError:
    pass

print("build_poly_from_exp and exp_from_monomial tests passed.")

NameError: name 'build_poly_from_exp' is not defined

# Test `compute_basis_exps`
Test the structure and output types of basis computation functions.

In [ ]:
# Test: compute_basis and compute_basis_polys
# Use small parameters for quick checks
a, b, c, N = 2, 1, 1, 2
exps_0, exps_1, exps_2 = compute_basis(a, b, c, N)

# Check keys and value types
assert isinstance(exps_0, dict) and isinstance(exps_1, dict) and isinstance(exps_2, dict)
assert all(isinstance(v, list) for v in exps_0.values())

# Check that exps_0[0] contains exponent lists of length 7
if exps_0[0]:
    assert all(len(e) == 7 for e in exps_0[0])

print("compute_basis tests passed.")

# Test `sub_x_for_u`, `sub_y_for_v`, and `reduce_poly`
Test the substitution and reduction functions for correctness.

In [ ]:
# Test: sub_x_for_u, sub_y_for_v, reduce_poly
R = base_ring
x, y, u, v, dx, dy = R.gens()

# x^a should become u
f = x**2
sub_x = sub_x_for_u(f, 2, R=R)
assert sub_x == u, f"sub_x_for_u failed: {sub_x} != {u}"

# y^c should become v
f = y**3
sub_y = sub_y_for_v(f, 1, 3, R=R)
assert sub_y == v, f"sub_y_for_v failed: {sub_y} != {v}"

# Test reduce_poly for a composition
f = x**2 + y**3
reduced = reduce_poly(f, 2, 1, 3)
assert u in reduced.variables() or v in reduced.variables(), "reduce_poly did not substitute as expected"

print("sub_x_for_u, sub_y_for_v, and reduce_poly tests passed.")

# Test `diff_from_exp_to_exp` and `diff_function`
Test the differential computation functions for expected output and structure.

In [ ]:
# Test: diff_from_exp_to_exp and diff_function
exp = [1, 2, 0, 0, 0, 0, 0]  # x^2
# Should produce a differential with respect to x
out = diff_from_exp_to_exp(exp, 2, 1, 1)
assert isinstance(out, list) and all(isinstance(e, list) for e in out), "diff_from_exp_to_exp output structure error"

# Test diff_function on a simple monomial
R = base_ring
x, y, u, v, dx, dy = R.gens()
f = x**2
poly_diff = diff_function(f, 2, 1, 1)
# Should be a polynomial (possibly zero)
assert hasattr(poly_diff, 'parent'), "diff_function did not return a polynomial"

print("diff_from_exp_to_exp and diff_function tests passed.")

# Test `matrix_of` and `torsion_exponents_from_elem_divs`
Test the matrix computation and torsion exponent extraction functions.

In [ ]:
# Test: matrix_of and torsion_exponents_from_elem_divs
# Use a small basis and identity function for simplicity
R = base_ring
x, y, u, v, dx, dy = R.gens()
basis = [x, y]
def identity_func(f):
    return f
M = matrix_of(basis, basis, identity_func, R=R)
assert hasattr(M, 'nrows') and M.nrows() == 2 and M.ncols() == 2, "matrix_of did not return expected matrix size"

# Test torsion_exponents_from_elem_divs
exponents = torsion_exponents_from_elem_divs([0, 1, 9, 27], 3)
assert exponents == [2, 3], f"torsion_exponents_from_elem_divs failed: {exponents}"

print("matrix_of and torsion_exponents_from_elem_divs tests passed.")

# Test `Frob_i` and `faster_Frob`
Test the consistency of the two Frobenius implementations on small polynomials.

In [ ]:
# Test: Frob_i and faster_Frob
R = base_ring
x, y, u, v, dx, dy = R.gens()
p, a, b, c, i = 3, 2, 1, 1, 2
N = a * c * i + (b - 1) * (c - 1) - 1
f = x + y + u
frobenius_orig = cap_deg(Frob_i(f, p, i, a, b, c, R=R), a, b, c, N)
frobenius_fast = faster_Frob(f, p, i, a, b, c, N, R=R)
assert frobenius_orig == frobenius_fast, f"Frob_i and faster_Frob mismatch: {frobenius_orig} != {frobenius_fast}"

print("Frob_i and faster_Frob tests passed.")